# Esercitazione 3

## Tweet like (a) Trump

In [1]:
import pandas as pd
from nltk import word_tokenize, TreebankWordDetokenizer
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline

L'esercitazione prevede di usare il dataset contenete circa 300 tweet, estratti dal profilo di Donald Trump, e generare un language model basato su n-grammi.

In [2]:
df = pd.read_csv('03-tweets.csv') # Caricamento del dataset
display(df)

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,LOSER! https://t.co/p5imhMJqS1,05-18-2020 14:55:14,32295,135445,False,1262396333064892416
1,Twitter for iPhone,Most of the money raised by the RINO losers of...,05-05-2020 18:18:26,19706,82425,False,1257736426206031874
2,Twitter for iPhone,....because they don’t know how to win and the...,05-05-2020 04:46:34,12665,56868,False,1257532112233803782
3,Twitter for iPhone,....lost for Evan “McMuffin” McMullin (to me)....,05-05-2020 04:46:34,13855,62268,False,1257532114666508291
4,Twitter for iPhone,....get even for all of their many failures. Y...,05-05-2020 04:46:33,8122,33261,False,1257532110971318274
...,...,...,...,...,...,...,...
256,Twitter Web Client,.@Cher attacked @MittRomney. She is an average...,05-10-2012 15:10:23,715,465,False,200603697435246592
257,Twitter Web Client,Firing @lisalampanelli may have come as a surp...,05-07-2012 02:58:18,45,19,False,199332301463748609
258,Twitter Web Client,My @SquawkCNBC interview discussing why @MittR...,03-06-2012 17:07:51,32,9,False,177078050750599168
259,TweetDeck,I feel sorry for Rosie 's new partner in love ...,12-14-2011 16:45:55,667,463,False,146994336670822400


Per svolgere questo task siamo interessati solo alla colonna contenente i tweet. Di seguito si utilizzano le funzioni della libreria pandas per estrarre i dati rilevanti ed effettuare la tokenizzazione.

In [3]:
trump_corpus = list(df['text'].apply(word_tokenize))

Per avere una rudimentale idea dei dati trattati, calcoliamo il numero medio di parole presenti in un tweet e di caratteri utilizzati.

In [4]:
avg_len, avg_word = sum([len(t) for t in df['text']]) // len(df), sum(len(t.split()) for t in df['text']) // len(df)
avg_len, avg_word

(163, 26)

Il parametro N indica la dimensione dell'n-gramma, in questo caso inizializzato a due.

In questo modo verrà creata una tabella per stimare la probabilità di emissione di una parola data la precedente.

In [5]:
N = 2

Il metodo *padded_everygram_pipeline* permette di ottenere gli n-grammi per il modello e il vocabolario estratto dai dati, inoltre inserisce automaticamente i tag per effettuare il padding in modo da stimare correttamente anche come iniziare e concludere una frase.

In [6]:
n_grams, vocabulary = padded_everygram_pipeline(N, trump_corpus)

L'addestramento di un language model basato su n-grammi si riduce al calcolo delle probabilità di emissione.

Per calcolare tale matrice si usa il classico approccio dei modelli basati su n-grammi, al quale si applica la correzione di Laplace.

In [7]:
trump_model = Laplace(N)
trump_model.fit(n_grams, vocabulary)

Il metodo *generate_sent* permette di generare una frase specificandone la lunghezza, rimuovendo i tag per il padding in modo da rendere verosimile l'output.

In [10]:
def generate_sent(model, random_seed, num_words):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return TreebankWordDetokenizer().detokenize(content)

In [24]:
print(generate_sent(trump_model, 5, num_words=avg_word))

in race that you in the American dream!


In modo analogo, il modello può essere addestrato con valori di N variabili, a seguire un esempio con N = 3.

In [25]:
N = 3
n_grams, vocabulary = padded_everygram_pipeline(N, trump_corpus)
trump_model = Laplace(N)
trump_model.fit(n_grams, vocabulary)

print(generate_sent(trump_model, 5, num_words=avg_word))

highest -and you all know it!


In sintesi, i modelli linguistici basati su n-grammi sono estremamente veloci e semplici da addestrare. Tuttavia, rispetto a modelli più sofisticati, presentano evidenti limiti nella generazione di testi.

Tali limiti sono dovuti all'approccio puramente statistico, che si limita a produrre la n-esima parola data una "storia" di n-1 termini.